In [1]:
import pymongo
import pandas as pd
import numpy as np

from pymongo import MongoClient
from bson.objectid import ObjectId

import datetime

import matplotlib.pyplot as plt

from collections import defaultdict


%matplotlib inline
import json
plt.style.use('ggplot')

import seaborn as sns

from math import log10, floor

In [2]:
## Connect to local DB

client = MongoClient('localhost', 27017)
print ("Setup db access")

Setup db access


In [3]:
#
# Get collections from mongodb
#
#db = client.my_test_db
db = client.test


In [4]:
chunk = 100000
start = 0
end = start + chunk

In [5]:
reponses = db.anon_student_task_responses.find()[start:end]

In [6]:
df_responses = pd.DataFrame(list(reponses))

In [7]:
print (df_responses.head())

                        _id               behavioral_traits  bonus  correct  \
0  5a00f1739100de1a390000d0                              []  False     True   
1  5a00f1739100de1a390000d5  [measuring_tools, orange_tick]  False     True   
2  5a00f1739100de1a390000d9                              []  False     True   
3  5a00f1739100de1a390000dc                              []  False     True   
4  5a00f1739100de1a390000df                              []  False     True   

       diff          id  incomplete         lesson  \
0  0.000000  nvrm82_9Yv       False       nline_1b   
1  0.563288  jVG3p9f-20       False       nline_1b   
2  0.601043  _NUUDSBMum       False  equivalence_0   
3  0.686276  B6HmMEMpoL       False  equivalence_0   
4  0.642014  IYWiIP26on       False  equivalence_0   

                                       level_summary  \
0  {'entered': True, 'path': 'nline_1b', 'lm_stat...   
1  {'entered': True, 'path': 'nline_1b', 'lm_stat...   
2  {'subject': 'fractions', 'uni

In [8]:
df2 = df_responses.join(pd.DataFrame(df_responses["student"].to_dict()).T)

In [9]:
df2 = df2.join(pd.DataFrame(df2['level_summary'].to_dict()).T)

In [10]:
df2 = df2.join(pd.DataFrame(df2['problems'].to_dict()).T)

In [11]:
df3  = df2.copy()

In [12]:
## Look act columns
print (df_responses.columns)

Index(['_id', 'behavioral_traits', 'bonus', 'correct', 'diff', 'id',
       'incomplete', 'lesson', 'level_summary', 'problem_set',
       'problem_set_id', 'problem_set_subspace', 'qual_id',
       'randomly_selected', 'response', 'response_idx', 'retried',
       'screenshot_url', 'second_try', 'session_id', 'student', 'sublesson',
       't', 'time_spent', 'timestamp', 'txt', 'untouched'],
      dtype='object')


In [13]:
## How many data samples
print (len(df_responses), "Number of entries")

100000 Number of entries


In [14]:
## Make 'description' a feature wih important words mapped

In [15]:
df3.columns

Index(['_id', 'behavioral_traits', 'bonus', 'correct', 'diff', 'id',
       'incomplete', 'lesson', 'level_summary', 'problem_set',
       'problem_set_id', 'problem_set_subspace', 'qual_id',
       'randomly_selected', 'response', 'response_idx', 'retried',
       'screenshot_url', 'second_try', 'session_id', 'student', 'sublesson',
       't', 'time_spent', 'timestamp', 'txt', 'untouched', 'grade',
       'mvt_group', 'mvt_group_code', 'school_id', 'section_id', 'student_id',
       'blank_slate_mastery', 'description', 'entered', 'lesson_type',
       'lm_stats', 'mastery', 'name', 'path', 'problems', 'stars', 'subject',
       't_elapsed', 'time_entered', 'time_exited', 'title', 'type',
       'unit_name', 'unit_rank', 'nretry_right', 'nretry_wrong', 'nright',
       'ntotal', 'nuntouched', 'nwrong'],
      dtype='object')

In [16]:
df3['percent_correct'] = df3['nright'].astype(float) / df3['ntotal']

In [17]:
df3.iloc[0]

_id                                              5a00f1739100de1a390000d0
behavioral_traits                                                      []
bonus                                                               False
correct                                                              True
diff                                                                    0
id                                                             nvrm82_9Yv
incomplete                                                          False
lesson                                                           nline_1b
level_summary           {'entered': True, 'path': 'nline_1b', 'lm_stat...
problem_set             lessons/fractions/lesson13_1/part_a/media/prob...
problem_set_id                                                 _Pd6uU3viE
problem_set_subspace                                                parta
qual_id                                       _Pd6uU3viE.parta.nvrm82_9Yv
randomly_selected                     

In [20]:
'''
for idx in range(100):
    print ('index"', idx)
    print (df3.iloc[idx]['lesson'])
    print (df3.iloc[idx]['response'])
'''

'\nfor idx in range(100):\n    print (\'index"\', idx)\n    print (df3.iloc[idx][\'lesson\'])\n    print (df3.iloc[idx][\'response\'])\n'

In [21]:
def stringify_response(resp):
    my_val = str(resp).replace("': ","_")
    my_val = my_val.replace("_{"," ")
    my_val = my_val.replace("_[",", ")
    for c in [']','[','{','}',"'","",","]:
        my_val = my_val.replace(c,'')
    return my_val


In [22]:
stringify_response(df3.iloc[0]['response'])

'fraction_cblock_chains right_442 sum numerator_1 denominator_2 __as3_type_Fraction pieces 1/2 left_97 lcm_sum numerator_1 denominator_2 __as3_type_Fraction plain_image_groups total_1 url_assets/cms/wootmath_fractions/number_line/markers/end_marker_noline.swf total_1 url_assets/cms/wootmath_fractions/number_line/markers/start_marker.swf total_1 url_assets/cms/wootmath_fractions/number_line/objects/dog.swf total_1 url_assets/cms/wootmath_fractions/number_line/objects/cat_dog_trail.swf den_2 fraction_input_value_1/2 num_1 fraction_cblock_total_count_1 numberline_associations  fraction_cblock_counts 1/2_1 fraction_cblock_containment  whole_'

In [23]:
df3['response_str'] = df3['response'].apply(stringify_response)

In [24]:
for idx in range(20):
    print (idx, df3['response_str'].iloc[idx])

0 fraction_cblock_chains right_442 sum numerator_1 denominator_2 __as3_type_Fraction pieces 1/2 left_97 lcm_sum numerator_1 denominator_2 __as3_type_Fraction plain_image_groups total_1 url_assets/cms/wootmath_fractions/number_line/markers/end_marker_noline.swf total_1 url_assets/cms/wootmath_fractions/number_line/markers/start_marker.swf total_1 url_assets/cms/wootmath_fractions/number_line/objects/dog.swf total_1 url_assets/cms/wootmath_fractions/number_line/objects/cat_dog_trail.swf den_2 fraction_input_value_1/2 num_1 fraction_cblock_total_count_1 numberline_associations  fraction_cblock_counts 1/2_1 fraction_cblock_containment  whole_
1 fraction_cblock_total_count_4 plain_image_groups total_1 url_assets/cms/wootmath_fractions/number_line/objects/panda.swf total_1 url_assets/cms/wootmath_fractions/number_line/markers/start_marker.swf input_4 fraction_cblock_chains right_856 sum numerator_1 denominator_1 __as3_type_Fraction pieces 1/4 1/4 1/4 1/4 left_165 lcm_sum numerator_4 denomina

In [25]:
df3.columns

Index(['_id', 'behavioral_traits', 'bonus', 'correct', 'diff', 'id',
       'incomplete', 'lesson', 'level_summary', 'problem_set',
       'problem_set_id', 'problem_set_subspace', 'qual_id',
       'randomly_selected', 'response', 'response_idx', 'retried',
       'screenshot_url', 'second_try', 'session_id', 'student', 'sublesson',
       't', 'time_spent', 'timestamp', 'txt', 'untouched', 'grade',
       'mvt_group', 'mvt_group_code', 'school_id', 'section_id', 'student_id',
       'blank_slate_mastery', 'description', 'entered', 'lesson_type',
       'lm_stats', 'mastery', 'name', 'path', 'problems', 'stars', 'subject',
       't_elapsed', 'time_entered', 'time_exited', 'title', 'type',
       'unit_name', 'unit_rank', 'nretry_right', 'nretry_wrong', 'nright',
       'ntotal', 'nuntouched', 'nwrong', 'percent_correct', 'response_str'],
      dtype='object')

In [26]:
## In Response:
### convert K, V, and all K_V into words in a text doc
### Then add text
### The add description


In [27]:
def make_string_from_list(key, elem_list):
    # Append key to each item in list
    ans = ''
    for elem in elem_list:
        ans += key + '_' + elem 
        
        

        
    

def make_string(elem, key=None, top=True):
    ans = ''
    if not elem:
        return ans
    if top:
        top = False
        top_keys = []
        for idx in range(len(elem.keys())):
            top_keys.append(True)
            
    for idx, key in enumerate(elem.keys()):
        if top_keys[idx]:
            top = True
            top_keys[idx] = False
            ans += ' '
        else:
            top = False
        #print ('ans = ', ans)
        #print (type(elem[key]))
        if type(elem[key]) is str or\
                type(elem[key]) is int:
            #print ('add value', elem[key])
            value = str(elem[key])
            #ans += key + '_' + value + ' ' + value + ' '
            ans += key + '_' + value
        elif type(elem[key]) is list:
            #print ('add list', elem[key])
            temp_elem = dict()
            for item in elem[key]:
                temp_elem[key] = item
                ans += make_string(temp_elem, top) 
        elif type(elem[key]) is dict:
            #print ('add dict', elem[key])
            for item_key in elem[key].keys():
                temp_elem = dict()
                temp_elem[item_key] = elem[key][item_key]
                ans += key + '_' + make_string(temp_elem, top)
        elif type(elem[key]) is float:
            #print ('add dict', elem[key])
            sig = 2
            value = elem[key]
            value = round(value, sig-int(
            floor(log10(abs(value))))-1)
            value = str(value)
            #ans += key + '_' + value + ' ' + value + ' '
            ans += key + '_' + value + ' '
        # ans += ' ' + key + ' '
        #print ('not handled', elem[key])
     
        
    return ans
        
            
            

In [28]:
df3['response_doc'] = df3['response'].map(make_string)

In [29]:
df3['response_doc'] = df3['response_doc'].map(lambda x: x + ' ')

In [30]:
df3['response_doc'] = df3['response_doc'] + df3['txt'] 

In [31]:
df3['response_doc'] = df3['response_doc'].map(lambda x: x + ' ')

In [32]:
df3['response_doc'] = df3['response_doc'] +  df3['description']

In [33]:
df3['response_doc'] = df3['response_doc'].map(lambda x: x.replace("\n", ""))

In [34]:
df3['response_doc'] = df3['response_doc'].map(lambda x: x.replace("?", " "))

In [35]:
df3.iloc[100]['response_doc']

" radio_group_mc2_ choice_Aradio_group_mc2_ text_Yes radio_group_mc1_ choice_Bradio_group_mc1_ text_No  plain_image_groups_ url_assets/cms/wootmath_fractions/equal_parts/sixths/sixth_15.swfplain_image_groups_ total_1 plain_image_groups_ url_assets/cms/wootmath_fractions/equal_parts/sixths/sixth_06.swfplain_image_groups_ total_1 Do these shapes have 5/6 shaded Answer: [object Object] Students deepen their understanding of fractions by identifying area models that are divided into equal parts and those that aren't. "

In [36]:
df3.iloc[100]['response']

{'plain_image_groups': [{'total': 1,
   'url': 'assets/cms/wootmath_fractions/equal_parts/sixths/sixth_15.swf'},
  {'total': 1,
   'url': 'assets/cms/wootmath_fractions/equal_parts/sixths/sixth_06.swf'}],
 'radio_group_mc1': {'choice': 'B', 'text': 'No'},
 'radio_group_mc2': {'choice': 'A', 'text': 'Yes'}}

In [37]:
for idx in range(20):
    print (idx, df3['response_doc'].iloc[idx])

0   fraction_cblock_chains_ right_442fraction_cblock_chains_ sum_ numerator_1sum_ denominator_2sum_ __as3_type_Fractionfraction_cblock_chains_  pieces_1/2fraction_cblock_chains_ left_97fraction_cblock_chains_ lcm_sum_ numerator_1lcm_sum_ denominator_2lcm_sum_ __as3_type_Fraction  plain_image_groups_ total_1plain_image_groups_ url_assets/cms/wootmath_fractions/number_line/markers/end_marker_noline.swf plain_image_groups_ total_1plain_image_groups_ url_assets/cms/wootmath_fractions/number_line/markers/start_marker.swf plain_image_groups_ total_1plain_image_groups_ url_assets/cms/wootmath_fractions/number_line/objects/dog.swf plain_image_groups_ total_1plain_image_groups_ url_assets/cms/wootmath_fractions/number_line/objects/cat_dog_trail.swf den_2 fraction_input_value_1/2 num_1 fraction_cblock_total_count_1   fraction_cblock_counts_ 1/2_1  whole_ Use the 1/2 pieces to figure out how far the dog traveled.Answer: 1/2 In the first part of this lesson, student partition the number line into 

In [38]:
df3['response_doc'] = df3['response_doc'].map( lambda x: x + ' ' + " ".join(x.split('/')) if '/' in x else x)

In [39]:
df3.iloc[100]['response_doc']

" radio_group_mc2_ choice_Aradio_group_mc2_ text_Yes radio_group_mc1_ choice_Bradio_group_mc1_ text_No  plain_image_groups_ url_assets/cms/wootmath_fractions/equal_parts/sixths/sixth_15.swfplain_image_groups_ total_1 plain_image_groups_ url_assets/cms/wootmath_fractions/equal_parts/sixths/sixth_06.swfplain_image_groups_ total_1 Do these shapes have 5/6 shaded Answer: [object Object] Students deepen their understanding of fractions by identifying area models that are divided into equal parts and those that aren't.   radio_group_mc2_ choice_Aradio_group_mc2_ text_Yes radio_group_mc1_ choice_Bradio_group_mc1_ text_No  plain_image_groups_ url_assets cms wootmath_fractions equal_parts sixths sixth_15.swfplain_image_groups_ total_1 plain_image_groups_ url_assets cms wootmath_fractions equal_parts sixths sixth_06.swfplain_image_groups_ total_1 Do these shapes have 5 6 shaded Answer: [object Object] Students deepen their understanding of fractions by identifying area models that are divided in

In [40]:
df3['response_doc'] = df3['response_doc'].map( lambda x: x.replace('[',' '))
df3['response_doc'] = df3['response_doc'].map( lambda x: x.replace(']',' '))

In [41]:
df3.iloc[100]['response_doc']

" radio_group_mc2_ choice_Aradio_group_mc2_ text_Yes radio_group_mc1_ choice_Bradio_group_mc1_ text_No  plain_image_groups_ url_assets/cms/wootmath_fractions/equal_parts/sixths/sixth_15.swfplain_image_groups_ total_1 plain_image_groups_ url_assets/cms/wootmath_fractions/equal_parts/sixths/sixth_06.swfplain_image_groups_ total_1 Do these shapes have 5/6 shaded Answer:  object Object  Students deepen their understanding of fractions by identifying area models that are divided into equal parts and those that aren't.   radio_group_mc2_ choice_Aradio_group_mc2_ text_Yes radio_group_mc1_ choice_Bradio_group_mc1_ text_No  plain_image_groups_ url_assets cms wootmath_fractions equal_parts sixths sixth_15.swfplain_image_groups_ total_1 plain_image_groups_ url_assets cms wootmath_fractions equal_parts sixths sixth_06.swfplain_image_groups_ total_1 Do these shapes have 5 6 shaded Answer:  object Object  Students deepen their understanding of fractions by identifying area models that are divided in

In [42]:
docs = list(df3['response_doc'])

In [43]:
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [44]:
data_samples = docs

In [45]:
n_features = 1000
n_samples = len(data_samples)
n_topics = 10
n_top_words = 20

In [46]:
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')

Extracting tf-idf features for NMF...


In [47]:
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


done in 14.377s.


In [48]:
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


Extracting tf features for LDA...
done in 14.297s.


In [49]:
# Fit the NMF model
print("Fitting the NMF model with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_topics, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

Fitting the NMF model with tf-idf features, n_samples=100000 and n_features=1000...
done in 15.776s.


In [50]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [51]:
print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)


Topics in NMF model:
Topic #0:
__as3_type_fractionfraction_cblock_chains_ fraction_cblock_chains_ sum_ lcm_sum_ numerator_1lcm_sum_ numerator_1sum_ pieces_1 __as3_type_fraction denominator_1sum_ denominator_1lcm_sum_ pieces_1fraction_cblock_chains_ left_175fraction_cblock_chains_ bar1_ bar fraction_cblock_counts_ fraction_cblock_containment_ denominator_2sum_ denominator_4lcm_sum_ denominator_8lcm_sum_ denominator_8sum_
Topic #1:
pieces_1 fraction_circle_groups_ fraction chains_ scale_0 1_ pieces_1fraction_circle_groups_ 2_ fraction_circle_containment_ __as3_type_fraction 15 lcm_sum_ sum_ fraction_circle_counts_ pieces scale_1 1_1fraction_circle_counts_ circle y_300fraction_circle_groups_ unit1_
Topic #2:
cms wootmath_fractions plain_image_groups_ url_assets number_line swf total_1plain_image_groups_ markers objects_v2 objects traveled start_marker distance swfplain_image_groups_ total_1 line juice real world represent
Topic #3:
common numerators having comparison denominators match f

In [52]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

Fitting LDA models with tf features, n_samples=100000 and n_features=1000...


/Users/brianmckean/anaconda2/envs/hwenv/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


done in 150.299s.


In [53]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0:
pieces_1 12 fraction 10 15 __as3_type_fraction fraction_circle_groups_ pieces sum_ lcm_sum_ 1_ chains_ 2_ circle numerator_1sum_ fraction_circle_containment_ answer fraction_circle_counts_ scale_1 students
Topic #1:
url_assets plain_image_groups_ wootmath_fractions cms swf number_line total_1plain_image_groups_ markers objects traveled start_marker objects_v2 line number distance answer students fraction use far
Topic #2:
review whole_ answer fraction fractions lessons earlier selected following topics material mixed students ordering enter fraction_input_value_1 num_1 lesson form fractional
Topic #3:
students answer fractions using models numerator denominator correct visual model lesson comparison input_ benchmark grid compare equivalent choose fraction notation
Topic #4:
common fraction fractions students order denominators numerators math answer comparison compare correct bitmap_text_interp_ bitmap_text_inputs_ having enter fraction_input_value_ choo

In [54]:
n_features = 1000
n_samples = len(data_samples)
n_topics = 100
n_top_words = 20

In [55]:
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english',
                                      ngram_range(1,4))
t0 = time()
tfidf_matrix = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


SyntaxError: positional argument follows keyword argument (<ipython-input-55-efb0327faec2>, line 5)

In [ ]:
print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)



In [ ]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [ ]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()